<a href="https://colab.research.google.com/github/buganart/stylegan2-ada-pytorch/blob/main/stylegan2ada_generate.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#@markdown Before starting please save the notebook in your drive by clicking on `File -> Save a copy in drive`

In [ ]:
#@markdown Check GPU, should be a Tesla V100
!nvidia-smi -L
import os
print(f"We have {os.cpu_count()} CPU cores.")

In [ ]:
#@markdown Mount google drive
from google.colab import drive
from google.colab import output
drive.mount('/content/drive')

from pathlib import Path
if not Path("/content/drive/My Drive/IRCMS_GAN_collaborative_database").exists():
    raise RuntimeError(
        "Shortcut to our shared drive folder doesn't exits.\n\n"
        "\t1. Go to the google drive web UI\n"
        "\t2. Right click shared folder IRCMS_GAN_collaborative_database and click \"Add shortcut to Drive\""
    )

def clear_on_success(msg="Ok!"):
    if _exit_code == 0:
        output.clear()
        print(msg)

In [ ]:
#@markdown Install wandb and log in
%pip install wandb
output.clear()
import wandb
from pathlib import Path
wandb_drive_netrc_path = Path("drive/My Drive/colab/.netrc")
wandb_local_netrc_path = Path("/root/.netrc")
if wandb_drive_netrc_path.exists():
    import shutil

    print("Wandb .netrc file found, will use that to log in.")
    shutil.copy(wandb_drive_netrc_path, wandb_local_netrc_path)
else:
    print(
        f"Wandb config not found at {wandb_drive_netrc_path}.\n"
        f"Using manual login.\n\n"
        f"To use auto login in the future, finish the manual login first and then run:\n\n"
        f"\t!mkdir -p '{wandb_drive_netrc_path.parent}'\n"
        f"\t!cp {wandb_local_netrc_path} '{wandb_drive_netrc_path}'\n\n"
        f"Then that file will be used to login next time.\n"
    )

!wandb login
output.clear()
print("ok!")

# Description


In [ ]:
#@title Configuration

#@markdown Directories can be found via file explorer on the left by navigating into `drive` to the desired folders. 
#@markdown Then right-click and `Copy path`.

#@markdown Load Model
#@markdown - option1: set wandb run id of logged run to resume from there
resume_id = "1dsg4dbq" #@param {type:"string"}
#@markdown - option2: load checkpoint pkl file from the path
ckpt_file = "" #@param {type:"string"}

#@markdown Generate script option
#@markdown - Truncation psi 
latent = True #@param {type:"boolean"}
#@markdown - List of random seeds 
seeds = "1,5" #@param {type:"string"}
#@markdown - Truncation psi 
truncation_psi = 1 #@param {type:"number"}
#@markdown - number of samples per seed 
frames =  10#@param {type:"integer"}
#@markdown - Class label (unconditional if not specified)
class_idx = 0 #@param {type:"integer"}
#@markdown - Noise mode
noise_mode = "const" #@param ["const", "random", "none"]
#@markdown - The output directory of experiment data 
out_dir = "/content/drive/My Drive/IRCMS_GAN_collaborative_database/Experiments/colab-stylegan2" #@param {type:"string"}


if not ckpt_file and not resume_id:
    raise Exception("set one of ckpt_file or resume_id to load model.")

In [ ]:
#@title Clone stylegan2 repo

!git clone https://github.com/buganart/stylegan2-ada-pytorch

In [ ]:
#@title Install Dependencies

%cd /content/stylegan2-ada-pytorch
# %pip install -r requirements.txt
%pip install wandb
%pip install ninja
%pip install pyspng
clear_on_success("Dependencies installed.")

In [ ]:
#@title preprocess argument
out_dir = Path(out_dir)
out_dir.mkdir(parents=True, exist_ok=True)


run_dir = Path("tmp/run")
run_dir.mkdir(parents=True, exist_ok=True)


if resume_id:
    run = wandb.init(
            project="stylegan2",
            id=resume_id,
            entity="bugan",
            resume="allow",
            dir=run_dir,
            mode="offline",
        )
    model = wandb.restore("model.pkl")
    ckpt_file = model.name

clear_on_success("ok")


print("out_dir", out_dir)
print("ckpt_file", ckpt_file)

In [ ]:
#@title GENERATE

!python generate.py \
--outdir "$out_dir" \
--network "$ckpt_file" \
--latent "$latent" \
--frames "$frames" \
--seeds "$seeds" \
--trunc "$truncation_psi" \
--class "$class_idx" \
--noise-mode "$noise_mode"
 